## Tuklasin ang K-Means clustering gamit ang R at mga prinsipyo ng Tidy data.

### [**Pre-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

Sa araling ito, matututuhan mo kung paano lumikha ng mga cluster gamit ang Tidymodels package at iba pang mga package sa ecosystem ng R (tatawagin natin silang mga kaibigan 🧑‍🤝‍🧑), at ang Nigerian music dataset na na-import mo kanina. Tatalakayin natin ang mga pangunahing kaalaman ng K-Means para sa Clustering. Tandaan na, tulad ng natutunan mo sa nakaraang aralin, maraming paraan upang magtrabaho sa mga cluster at ang pamamaraan na gagamitin mo ay nakadepende sa iyong data. Susubukan natin ang K-Means dahil ito ang pinakakaraniwang clustering technique. Tara na!

Mga terminong matututuhan mo:

-   Silhouette scoring

-   Elbow method

-   Inertia

-   Variance

### **Panimula**

Ang [K-Means Clustering](https://wikipedia.org/wiki/K-means_clustering) ay isang pamamaraan na nagmula sa larangan ng signal processing. Ginagamit ito upang hatiin at i-partition ang mga grupo ng data sa `k clusters` batay sa pagkakapareho ng kanilang mga katangian.

Ang mga cluster ay maaaring i-visualize bilang [Voronoi diagrams](https://wikipedia.org/wiki/Voronoi_diagram), na binubuo ng isang punto (o 'seed') at ang kaukulang rehiyon nito.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infographic ni Jen Looper</figcaption>

Ang K-Means clustering ay may mga sumusunod na hakbang:

1.  Sinisimulan ng data scientist sa pamamagitan ng pagtukoy ng nais na bilang ng mga cluster na gagawin.

2.  Susunod, ang algorithm ay random na pumipili ng K na obserbasyon mula sa data set upang magsilbing mga paunang sentro para sa mga cluster (i.e., centroids).

3.  Pagkatapos, ang bawat natitirang obserbasyon ay itinalaga sa pinakamalapit na centroid.

4.  Susunod, kinakalkula ang bagong mean ng bawat cluster at ang centroid ay inilipat sa mean.

5.  Ngayon na ang mga sentro ay muling na-kalkula, ang bawat obserbasyon ay muling sinusuri upang makita kung mas malapit ito sa ibang cluster. Ang lahat ng mga bagay ay muling itinalaga gamit ang na-update na mga mean ng cluster. Ang mga hakbang ng cluster assignment at centroid update ay paulit-ulit na inuulit hanggang sa ang mga assignment ng cluster ay tumigil sa pagbabago (i.e., kapag naabot ang convergence). Karaniwan, ang algorithm ay nagtatapos kapag ang bawat bagong iteration ay nagreresulta sa napakaliit na paggalaw ng mga centroid at ang mga cluster ay nagiging static.

<div>

> Tandaan na dahil sa randomization ng mga paunang k obserbasyon na ginamit bilang mga panimulang centroid, maaari tayong makakuha ng bahagyang magkaibang resulta sa bawat pagkakataon na ilalapat natin ang pamamaraan. Dahil dito, karamihan sa mga algorithm ay gumagamit ng ilang *random starts* at pinipili ang iteration na may pinakamababang WCSS. Kaya't mariing inirerekomenda na palaging patakbuhin ang K-Means gamit ang ilang halaga ng *nstart* upang maiwasan ang *hindi kanais-nais na local optimum.*

</div>

Ang maikling animation na ito gamit ang [artwork](https://github.com/allisonhorst/stats-illustrations) ni Allison Horst ay nagpapaliwanag ng proseso ng clustering:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Artwork ni @allison_horst</figcaption>

Isang mahalagang tanong na lumalabas sa clustering ay ito: paano mo malalaman kung ilang cluster ang dapat mong paghiwalayin ang iyong data? Isa sa mga kahinaan ng paggamit ng K-Means ay ang pangangailangan na tukuyin ang `k`, o ang bilang ng mga `centroids`. Sa kabutihang-palad, ang `elbow method` ay tumutulong upang tantyahin ang isang magandang panimulang halaga para sa `k`. Susubukan mo ito sa ilang sandali.

### 

**Paunang Kaalaman**

Ipagpapatuloy natin mula sa kung saan tayo tumigil sa [nakaraang aralin](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), kung saan sinuri natin ang data set, gumawa ng maraming visualization, at na-filter ang data set sa mga obserbasyong mahalaga. Siguraduhing balikan ito!

Kakailanganin natin ng ilang mga package upang matapos ang module na ito. Maaari mo silang mai-install gamit ang: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Bilang alternatibo, ang script sa ibaba ay sinusuri kung mayroon ka ng mga kinakailangang package upang makumpleto ang module na ito at awtomatikong ini-install ang mga nawawala.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Tara, simulan na natin!

## 1. Isang sayaw kasama ang datos: Piliin ang 3 pinakapopular na genre ng musika

Ito ay isang pagbalik-tanaw sa ginawa natin sa nakaraang aralin. Halika't himayin natin ang datos!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Ang galing nun!

## 2. Mas malalim na pagsusuri ng datos.

Gaano kalinis ang datos na ito? Tingnan natin kung may mga outlier gamit ang mga box plot. Magtuon tayo sa mga numeric na column na may mas kaunting outlier (bagamat maaari mong linisin ang mga outlier). Ang mga box plot ay maaaring magpakita ng saklaw ng datos at makakatulong sa pagpili kung aling mga column ang gagamitin. Tandaan, ang mga box plot ay hindi nagpapakita ng variance, isang mahalagang elemento ng mahusay na clusterable na datos. Mangyaring tingnan ang [talakayang ito](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) para sa karagdagang impormasyon.

Ang mga [Boxplot](https://en.wikipedia.org/wiki/Box_plot) ay ginagamit upang ipakita nang grapikal ang distribusyon ng `numeric` na datos, kaya magsimula tayo sa *pagpili* ng lahat ng numeric na column kasama ang mga sikat na genre ng musika.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Tingnan kung paano pinapadali ng selection helper na `where` ang prosesong ito 💁? Tuklasin ang iba pang mga function na tulad nito [dito](https://tidyselect.r-lib.org/).

Dahil gagawa tayo ng boxplot para sa bawat numeric na katangian at nais nating iwasan ang paggamit ng mga loop, ire-reformat natin ang ating data sa isang *mas mahaba* na format na magpapahintulot sa atin na magamit ang `facets` - mga subplot na nagpapakita ng bawat subset ng data.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Mas mahaba! Oras na para sa ilang `ggplots`! Kaya anong `geom` ang gagamitin natin?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Easy-gg!

Ngayon makikita natin na medyo magulo ang datos: sa pamamagitan ng pag-obserba sa bawat column gamit ang boxplot, makikita mo ang mga outlier. Pwede mong suriin ang dataset at alisin ang mga outlier na ito, pero magiging masyadong limitado ang datos kung gagawin iyon.

Sa ngayon, piliin natin kung aling mga column ang gagamitin para sa ating clustering exercise. Piliin natin ang mga numeric na column na may magkatulad na range. Pwede nating i-encode ang `artist_top_genre` bilang numeric pero iiwan muna natin ito sa ngayon.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Pag-compute ng k-means clustering sa R

Maaaring i-compute ang k-means sa R gamit ang built-in na `kmeans` function, tingnan ang `help("kmeans()")`. Ang `kmeans()` function ay tumatanggap ng data frame na may lahat ng numeric na column bilang pangunahing argumento nito.

Ang unang hakbang sa paggamit ng k-means clustering ay ang pagtukoy ng bilang ng clusters (k) na bubuuin sa final na solusyon. Alam natin na mayroong 3 genre ng kanta na nakuha mula sa dataset, kaya subukan natin ang 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Ang kmeans object ay naglalaman ng ilang impormasyon na mahusay na ipinaliwanag sa `help("kmeans()")`. Sa ngayon, mag-focus tayo sa ilang bahagi nito. Nakikita natin na ang datos ay nahati sa 3 clusters na may mga sukat na 65, 110, 111. Ang output ay naglalaman din ng mga sentro ng cluster (means) para sa 3 grupo sa kabuuan ng 5 variables.

Ang clustering vector ay ang cluster assignment para sa bawat obserbasyon. Gamitin natin ang `augment` function upang idagdag ang cluster assignment sa orihinal na data set.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfecto, nahati na natin ang ating data set sa 3 grupo. Kaya, gaano nga ba kaganda ang ating clustering 🤷? Tingnan natin ang `Silhouette score`.

### **Silhouette score**

Ang [Silhouette analysis](https://en.wikipedia.org/wiki/Silhouette_(clustering)) ay maaaring gamitin upang pag-aralan ang distansya ng paghihiwalay sa pagitan ng mga nabuong cluster. Ang score na ito ay nagbabago mula -1 hanggang 1, at kung ang score ay malapit sa 1, ang cluster ay siksik at maayos na nahihiwalay mula sa ibang mga cluster. Ang halaga na malapit sa 0 ay kumakatawan sa mga cluster na nag-o-overlap, kung saan ang mga sample ay napakalapit sa decision boundary ng mga kalapit na cluster. [source](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Ang average silhouette method ay kinakalkula ang average silhouette ng mga obserbasyon para sa iba't ibang halaga ng *k*. Ang mataas na average silhouette score ay nagpapahiwatig ng magandang clustering.

Ang `silhouette` function sa cluster package ay ginagamit upang kalkulahin ang average silhouette width.

> Ang silhouette ay maaaring kalkulahin gamit ang anumang [distance](https://en.wikipedia.org/wiki/Distance "Distance") metric, tulad ng [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") o [Manhattan distance](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") na tinalakay natin sa [nakaraang aralin](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Ang ating score ay **.549**, kaya nasa gitna lang. Ipinapakita nito na ang ating data ay hindi gaanong angkop para sa ganitong uri ng clustering. Tingnan natin kung makukumpirma natin ang hinalang ito sa pamamagitan ng visual. Ang [factoextra package](https://rpkgs.datanovia.com/factoextra/index.html) ay nagbibigay ng mga function (`fviz_cluster()`) para i-visualize ang clustering.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Ang overlap sa mga cluster ay nagpapahiwatig na ang ating datos ay hindi gaanong angkop para sa ganitong uri ng clustering, ngunit magpatuloy tayo.

## 4. Pagtukoy sa pinakamainam na bilang ng mga cluster

Isang mahalagang tanong na madalas lumalabas sa K-Means clustering ay ito - kung walang kilalang mga label ng klase, paano mo malalaman kung ilang cluster ang dapat paghiwalayin ang iyong datos?

Isang paraan upang malaman ito ay ang paggamit ng isang sample ng datos upang `lumikha ng serye ng mga clustering model` na may dumaraming bilang ng mga cluster (hal. mula 1-10), at suriin ang mga clustering metric tulad ng **Silhouette score.**

Tukuyin natin ang pinakamainam na bilang ng mga cluster sa pamamagitan ng pagkalkula ng clustering algorithm para sa iba't ibang halaga ng *k* at pagsusuri sa **Within Cluster Sum of Squares** (WCSS). Ang kabuuang within-cluster sum of square (WCSS) ay sumusukat sa pagkakabuklod ng clustering, at nais natin itong maging kasing-liit hangga't maaari, kung saan ang mas mababang halaga ay nangangahulugang mas malapit ang mga datos.

Suriin natin ang epekto ng iba't ibang pagpipilian ng `k`, mula 1 hanggang 10, sa clustering na ito.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Ngayon na mayroon na tayong kabuuang within-cluster sum-of-squares (tot.withinss) para sa bawat clustering algorithm na may center *k*, gagamitin natin ang [elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) upang mahanap ang pinakamainam na bilang ng clusters. Ang pamamaraang ito ay binubuo ng pag-plot ng WCSS bilang isang function ng bilang ng clusters, at pagpili sa [elbow ng kurba](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") bilang bilang ng clusters na gagamitin.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Ang grap ay nagpapakita ng malaking pagbaba sa WCSS (kaya mas *mahigpit*) habang ang bilang ng mga cluster ay tumataas mula isa hanggang dalawa, at isa pang kapansin-pansing pagbaba mula dalawa hanggang tatlong cluster. Pagkatapos nito, ang pagbaba ay hindi na gaanong kapansin-pansin, na nagreresulta sa isang `elbow` 💪 sa chart sa bandang tatlong cluster. Ito ay isang magandang indikasyon na may dalawa hanggang tatlong makatuwirang hiwalay na mga cluster ng mga data point.

Maaari na nating kunin ang clustering model kung saan `k = 3`:

> `pull()`: ginagamit upang kunin ang isang column
>
> `pluck()`: ginagamit upang i-index ang mga data structure tulad ng mga list


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Sige! Tingnan natin ang mga cluster na nakuha. Gusto mo ba ng kaunting interaktibidad gamit ang `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Maaaring inaasahan natin na ang bawat cluster (kinakatawan ng iba't ibang kulay) ay magkakaroon ng magkakaibang genre (kinakatawan ng iba't ibang hugis).

Tingnan natin ang katumpakan ng modelo.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Ang katumpakan ng modelong ito ay hindi masama, ngunit hindi rin magaling. Maaaring ang datos ay hindi angkop para sa K-Means Clustering. Ang datos na ito ay masyadong hindi balanse, masyadong kaunti ang kaugnayan, at masyadong malaki ang pagkakaiba-iba sa pagitan ng mga halaga ng column para makabuo ng maayos na cluster. Sa katunayan, ang mga cluster na nabuo ay malamang na malaki ang impluwensya o pagkiling mula sa tatlong kategorya ng genre na tinukoy natin sa itaas.

Gayunpaman, ito ay isang magandang proseso ng pagkatuto!

Sa dokumentasyon ng Scikit-learn, makikita mo na ang isang modelong tulad nito, na may mga cluster na hindi masyadong malinaw ang pagkakahati, ay may problema sa 'variance':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infographic mula sa Scikit-learn</figcaption>



## **Variance**

Ang variance ay tinutukoy bilang "ang average ng mga squared differences mula sa Mean" [source](https://www.mathsisfun.com/data/standard-deviation.html). Sa konteksto ng problemang clustering na ito, tumutukoy ito sa datos kung saan ang mga numero sa ating dataset ay may tendensiyang lumayo nang sobra mula sa mean.

✅ Ito ay isang magandang pagkakataon para pag-isipan ang lahat ng paraan kung paano mo maaayos ang isyung ito. Ayusin pa ang datos? Gumamit ng ibang mga column? Gumamit ng ibang algorithm? Pahiwatig: Subukang [i-scale ang iyong datos](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) upang gawing normal ito at subukan ang ibang mga column.

> Subukan ang '[variance calculator](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' upang mas maunawaan ang konsepto.

------------------------------------------------------------------------

## **🚀Hamunin**

Maglaan ng oras sa notebook na ito, at ayusin ang mga parameter. Kaya mo bang mapabuti ang katumpakan ng modelo sa pamamagitan ng mas maayos na paglilinis ng datos (halimbawa, pag-aalis ng mga outlier)? Maaari kang gumamit ng weights upang bigyan ng mas malaking timbang ang ilang data samples. Ano pa ang maaari mong gawin upang makabuo ng mas magagandang cluster?

Pahiwatig: Subukang i-scale ang iyong datos. Mayroong commented code sa notebook na nagdadagdag ng standard scaling upang gawing mas magkatulad ang mga column ng datos sa kanilang saklaw. Mapapansin mo na habang bumababa ang silhouette score, ang 'kink' sa elbow graph ay nagiging mas maayos. Ito ay dahil ang hindi pag-scale sa datos ay nagbibigay-daan sa datos na may mas kaunting variance na magkaroon ng mas malaking timbang. Basahin pa ang tungkol sa problemang ito [dito](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Post-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Review & Self Study**

-   Tingnan ang isang K-Means Simulator [tulad nito](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Maaari mong gamitin ang tool na ito upang i-visualize ang mga sample data points at tukuyin ang mga centroid nito. Maaari mong i-edit ang randomness ng datos, bilang ng mga cluster, at bilang ng mga centroid. Nakakatulong ba ito upang magkaroon ka ng ideya kung paano maaaring ma-grupo ang datos?

-   Tingnan din ang [handout na ito tungkol sa K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) mula sa Stanford.

Gusto mo bang subukan ang iyong bagong natutunang clustering skills sa mga dataset na angkop para sa K-Means clustering? Tingnan ang:

-   [Train and Evaluate Clustering Models](https://rpubs.com/eR_ic/clustering) gamit ang Tidymodels at mga kaibigan

-   [K-means Cluster Analysis](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-means clustering with tidy data principles](https://www.tidymodels.org/learn/statistics/k-means/)

## **Takdang-Aralin**

[Subukan ang iba't ibang clustering methods](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## SALAMAT SA:

[Jen Looper](https://www.twitter.com/jenlooper) para sa paglikha ng orihinal na Python na bersyon ng module na ito ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) para sa paglikha ng mga kamangha-manghang ilustrasyon na ginagawang mas welcoming at engaging ang R. Hanapin ang higit pang mga ilustrasyon sa kanyang [gallery](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Masayang Pag-aaral,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Artwork ni @allison_horst</figcaption>



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
